# Text generation with an RNN

Projext Objectives:

- Generate text using RNN.
- Creat a training examples and targets for text generation.
- Build a RNN model for serquence generation using Keras Subclassing.
- Create a text generator and evaluate the output.



This projext demonstrates how to generate text using character-based RNN. We will work with a dataset of Shakespeare's writing from Andrej Karpathy's [The Unreasonable Effectiveness of Recurrent Neural Networks](http://karpathy.github.io/2015/05/21/rnn-effectiveness/). Given a sequence of characters from this data to predict the next character in the sequence ("e"). Longer sequences of text can be generated by calling the model repeatedly.

Bellow is a sample output when the model in this tutorial trained for 30 epochs and started with prompt "Q"

<pre>
QUEENE:
I had thought thou hadst a Roman; for the oracle,
Thus by All bids the man against the word,
Which are so weak of care, by old care done;
Your children were in your holy love,
And the precipitation through the bleeding throne.

BISHOP OF ELY:
Marry, and will, my lord, to weep in such a one were prettiest;
Yet now I was adopted heir
Of the world's lamentable day,
To watch the next way with his father with his face?

ESCALUS:
The cause why then we are all resolved more sons.

VOLUMNIA:
O, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, it is no sin it should be dead,
And love and pale as any will to that word.

QUEEN ELIZABETH:
But how long have I heard the soul for this world,
And show his hands of life be proved to stand.

PETRUCHIO:
I say he look'd on, if I must be content
To stay him from the fatal of our country's bliss.
His lordship pluck'd from this sentence then for prey,
And then let us twain, being the moon,
were she such a case as fills m
</pre>


While some of the sentences are grammatical, most odo not make sense. The model has not learned the meaning of words, but there are some things to consider.

  * The  model is charcter-based. When training started the model did not know how to spell an English-word, or that words were even a unit of text.

  * The strucuter of the outputresembles a play-blocks of text generally begin with a speaker name, in all the capital letters similar to the dataset.

  * As demonstrated below, the model is trained on small batches of text (100 cahracters each), and is stil able to generate a longer sequence of text with coherent strucutre.

## Setup
Import TensorFlow and other libraries

In [ ]:
import os
import time
import numpy as np
import tensorflow as tf

### Download the Shakesoeare dataset

In [1]:
path_to_file = tf.keras.utils.get_file(
    'shakespeare.txt',
    'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt',
)

NameError: name 'tf' is not defined

### Read the Data

In [ ]:
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
print(f"Lenght of text: {len(text)} characters")

Lenght of text: 1115394 characters


Let's take a look at the first 250 characters in text

In [ ]:
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



Let's see how many unique characters are in our documnet

In [ ]:
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

65 unique characters


This number represents the tokens that the Neural Network will consume during training, and will be generating during use

## Process the text

#### Vectorize the text
Before training, we need to convert the strings to a numerical representation.

Using `tf.keras.layers.StringLookup` layer can convert each character into an numeric ID, it just needs the text to be split into tokens first

In [ ]:
example_text = ['stefan', 'nafets']

chars = tf.strings.unicode_split(example_text, input_encoding='UTF-8')
chars

<tf.RaggedTensor [[b's', b't', b'e', b'f', b'a', b'n'],
 [b'n', b'a', b'f', b'e', b't', b's']]>

Now create the `tf.keras.layers.StringLookup` layers, this will help us to tranform the character into numbers.

In [ ]:
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab),
    mask_token=None
)

it converts from tokens to character ID's

In [ ]:
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[58, 59, 44, 45, 40, 53],
 [53, 40, 45, 44, 59, 58]]>

Since the goal of this project is to generate text, it will be important to invert this representation and recover human-readable strings from it. To achieve this we can use `tf.keras.layers.StringLookup(......, invert=True)`.

NOTE: Here instade of pasing the original vocabulary henerated with `sorted(set(list))` use the `get_vocabulary()` method so the `tf.keras.layers.StringLookup` layer so that the `[UNK]` are set the same way.

In [ ]:
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(),
    invert=True,
    mask_token=None
)

This layer recovers the cahracters from the vectors of ID's, and returns them as a `tf.RaggedTensor` of characters

In [ ]:
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b's', b't', b'e', b'f', b'a', b'n'],
 [b'n', b'a', b'f', b'e', b't', b's']]>

You can use `tf.strings.reduce_join()` to join the characters back into strings


In [ ]:
tf.strings.reduce_join(chars, axis=1).numpy()

array([b'stefan', b'nafets'], dtype=object)

In [ ]:
# lets create a function that we can call later
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=0)

## The prediciton task

Given a character, or a sequence of characters, what is the most probable next character?

This is the task that we training the model to perform. The input to the model will be sequence of characters, and you train the model to predict the output-the following character at each time step.

Since RNN's maintain an internal state that depends on the prevours seen elements, given all the characters computed untill this moment, what is the next character?

## Create training examples and targets

Next devide the text into examples sequences. Each input sequence will containe `seq_length` characters from the text.

For each input sequence, the corresponding targets containe the same length of text, except shifted one character to the right.

So break the text into chunks of `seq_length+1`. For example, say `seq_length` is 4 and our text is 'Hello'. The input sequence would be 'Hell', and the target sequnce 'ello'.

First use the `tf.data.Dataset.from_tensor_slices` fucntion to convert text vector into a stream of character indices.

In [ ]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(1115394,), dtype=int64, numpy=array([19, 48, 57, ..., 46,  9,  1])>

In [ ]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [ ]:
for ids in ids_dataset.take(10):
  print(chars_from_ids(ids).numpy().decode('UTF-8'))

F
i
r
s
t
 
C
i
t
i


In [ ]:
seq_length = 100
examples_per_epoch = len(text) // (seq_length + 1)

The `batch` method lets you easily convert these individual characters to sequences of the desired size

In [ ]:
sequences = ids_dataset.batch(seq_length + 1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b'F' b'i' b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':'
 b'\n' b'B' b'e' b'f' b'o' b'r' b'e' b' ' b'w' b'e' b' ' b'p' b'r' b'o'
 b'c' b'e' b'e' b'd' b' ' b'a' b'n' b'y' b' ' b'f' b'u' b'r' b't' b'h'
 b'e' b'r' b',' b' ' b'h' b'e' b'a' b'r' b' ' b'm' b'e' b' ' b's' b'p'
 b'e' b'a' b'k' b'.' b'\n' b'\n' b'A' b'l' b'l' b':' b'\n' b'S' b'p' b'e'
 b'a' b'k' b',' b' ' b's' b'p' b'e' b'a' b'k' b'.' b'\n' b'\n' b'F' b'i'
 b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':' b'\n' b'Y'
 b'o' b'u' b' '], shape=(101,), dtype=string)


Is easier to see what this is doing if you join the tokens beack into strings

In [ ]:
for seq in sequences.take(5):
  print(text_from_ids(seq).numpy())

b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
b'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
b"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
b"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
b'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


For training we will need a dataset of `(input, label)` pairs. Where `input` and `label` are sequences. At each time stap the inpout is the current character and the label is the next character

The following function that takes a sequence as input, duplicates, and shifts it to align the input and label for each timestamp:

In [ ]:
def split_input_target(sequence):
  input_text = sequence[:-1]
  target_text = sequence[1:]
  return input_text, target_text

In [ ]:
split_input_target(list('Tensorflow'))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

We can see that in the frist row we have the input without the target and in the secound row we have the target without the label, which makes input label pairs

In [ ]:
dataset = sequences.map(split_input_target)

In [ ]:
for input_example, target_example in dataset.take(1):
  print('Input  :', text_from_ids(input_example).numpy())
  print('Target :', text_from_ids(target_example).numpy())

Input  : b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target : b'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


#### Create training batches
We used `tf.data` to split the text into manageable sequences. But before feeding this data into the model, we need to shuffle the data and pack it into batches

In [ ]:
# Batch size
BATCH_SIZE = 63

"""Buffer size to shuffle the dataset
(TF data is designed to work wiht possobly infinite sequences,
so it dosen't attempt to shuffle the entire sequence in memory. Instadfe
it maintaines a buffer is which it shuffles elements)"""
BUFFER_SIZE = 10000

dataset = (
    dataset.shuffle(BUFFER_SIZE) \
    .batch(BATCH_SIZE, drop_remainder=True) \
    .prefetch(tf.data.experimental.AUTOTUNE)
)
dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(63, 100), dtype=tf.int64, name=None), TensorSpec(shape=(63, 100), dtype=tf.int64, name=None))>

## Build The Model

This section defines the model as `keras.Model` sublcass.

The model have the following layers:

* `tf.keras.layers.Embedding`: The input layer. A trainable lookup label that will map each character-ID to a vector with `embedding_dim` dimensions.
* `tf.keras.layers.GRU`: A type of RNN with seize `units=rnn_units` (We can also use LSTM layer here)
* `tf.keras.layers.Debse`: The output layer, with `vocab_size` outputs. it outputs one logit for each character in vocabulary. There are the log-likehood of each character accoriding to the model.

In [ ]:
# Length of the vocabulart in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

The class bellow does the following::
  - We derive a class from the keras.Model
  - The constructor is used to define the layers of the model
  - We define the pass forward using the layers defined in the constructor
  

In [ ]:
class MyModel(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, rnn_units):
        super().__init__(self)

        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(
            rnn_units,
            return_sequences=True,
            return_state=True
        )
        self.dense = tf.keras.layers.Dense(vocab_size)

    def call(self, inputs, states=None, return_state=False, training=False):
        x = self.embedding(inputs, training=training)
        """since we are training a text generation model,
        we use the previous state, in training. If there is no state,
        then we initialize the state """
        if states is None:
            states = self.gru.get_initial_state(x)
        x, states = self.gru(x, initial_state=states, training=training)
        x = self.dense(x, training=training)

        if return_state:
            return x, states
        else:
            return x

In [ ]:
model = MyModel(
    # Be sure the vocabulary size matches the `StringLookup` layers.
    vocab_size = len(ids_from_chars.get_vocabulary()),
    embedding_dim = embedding_dim,
    rnn_units = rnn_units,
)

For each character the model looks up the embedding, runs GRU one timestamp with the embedding as input, and applies the dnese layer to generate logits predicting the top-likelihhood of the next character

### Try The Model
Now run the model to see that it behaves as expected.

Frist check the shape of the output.

In [ ]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(
        example_batch_predictions.shape,
        "# (batch_size, sequence_length, vocab_size)",
    )

(63, 100, 66) # (batch_size, sequence_length, vocab_size)


In the above example for sequence length of the input is `100` but the model can be run on inputs of any length


In [ ]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  16896     
                                                                 
 gru (GRU)                   multiple                  3938304   
                                                                 
 dense (Dense)               multiple                  67650     
                                                                 
Total params: 4,022,850
Trainable params: 4,022,850
Non-trainable params: 0
_________________________________________________________________


To get the actual prediction from the model we need to sample from the output distribution, to get actual character indicies. The distribution is defined by the logits over hte character vocabulary

Note: It is important to sample from the distribution as taking the argmax of the distribution can easily get the model stuck in a loop

Try it for the first example in the batch

In [ ]:
sampled_indices = tf.random.categorical(
    example_batch_predictions[0],
    num_samples = 1
)

sampled_indices = tf.squeeze(sampled_indices, axis=1).numpy()

This gives us, at each timestep of the next character index:

In [ ]:
sampled_indices

array([14, 34, 22, 37, 14,  8, 48, 25, 49, 30, 36, 37, 38, 26, 43, 53, 60,
       49, 24,  8, 25,  4,  6, 43, 35, 33, 12, 26, 62, 53, 30, 38,  2,  8,
        8, 25, 26, 46, 57,  5, 23, 39, 19,  8, 40,  9, 24, 47, 64, 65, 34,
       46, 46, 22, 32, 51, 53, 32, 26, 23, 23, 52, 50, 54, 44, 34, 52, 43,
        6, 44, 48, 52, 24, 29,  7, 23, 59, 58, 29, 11, 62, 16, 18, 27, 32,
       22, 28, 19, 12, 25, 49, 62, 17, 47, 57, 58, 19, 61, 18, 65])

Decode these to see the text predicted by this untrained model

In [ ]:
print('Input:', text_from_ids(input_example_batch[0]).numpy(), sep='\n')
print('Next Char Predictions:', text_from_ids(sampled_indices).numpy(), sep='\n')

Input:
b'rd and a villain:\nWhich to maintain I would allow him odds,\nAnd meet him, were I tied to run afoot\nE'
Next Char Predictions:
b"AUIXA-iLjQWXYMdnujK-L$'dVT;MwnQY --LMgr&JZF-a.KhyzUggISlnSMJJmkoeUmd'eimKP,JtsP:wCENSIOF;LjwDhrsFvEz"


From this output of the we can see that the model is untrained and don't understand the text and can't make accurate predictions

### Train The Model
At this point the problem can be treated as a standard classification problem. Given the previous RNN statem and the input this time step, predict the class of the next character

#### Attach an optimizer, and loss function
The standard `tf.keras.losses.sparse_categorical_crossentropy` loss funciton works in this case because it is applied across the last dimension of the predictions

Beacuse the model returns logits, you need to set the `from_logits` flag.


In [ ]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [ ]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print(
    'Prediciton shape: ',
    example_batch_predictions.shape,
    '# (batch_size, sequence_length, vocab_size',
    sep='\n'
)
print('New loss:        ', example_batch_mean_loss)

Prediciton shape: 
(63, 100, 66)
# (batch_size, sequence_length, vocab_size
New loss:         tf.Tensor(4.18988, shape=(), dtype=float32)


A newly initialized model should't be to sure of itself, the output logits should all have similar magnitudes. To confirm this you can chekc that the exmponential of the mean loss is approximatly equal to the vocabulary size. A much higher loss means the model is sure of it's wrong answers and is badly initialized

In [ ]:
tf.exp(example_batch_mean_loss).numpy()

66.01486

Note: This value can from the examples it was evaluated on

In [ ]:
model.compile(optimizer='adam', loss=loss)

#### Configure checkpoints
We will use `tf.keras.callbacks.ModelChekpoint` to ensure that checkpoints are saved during training.

In [ ]:
# Path where tha checkpoints will be saved
checkpoint_path = './training_checkpoints'
# Name of the checkpoint file
checkpoint_prefix = os.path.join(checkpoint_path, "ckpt_(epoch)")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath = checkpoint_prefix, save_weight_only=True
)

## Fit the Model
To keep training time resonable, uses 10 epochs to train the model.

Note: Set the runtime to GPU for faster training

In [ ]:
%%time
EPOCHS = 10
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/10
174/175 [============================>.] - ETA: 0s - loss: 2.6899

175/175 [==============================] - 26s 88ms/step - loss: 2.6868
Epoch 2/10
175/175 [==============================] - ETA: 0s - loss: 1.9593

175/175 [==============================] - 17s 83ms/step - loss: 1.9593
Epoch 3/10
175/175 [==============================] - ETA: 0s - loss: 1.6845

175/175 [==============================] - 19s 87ms/step - loss: 1.6845
Epoch 4/10
175/175 [==============================] - ETA: 0s - loss: 1.5293

175/175 [==============================] - 16s 76ms/step - loss: 1.5293
Epoch 5/10
175/175 [==============================] - ETA: 0s - loss: 1.4338

...

175/175 [==============================] - 15s 76ms/step - loss: 1.3672
Epoch 7/10
175/175 [==============================] - ETA: 0s - loss: 1.3156

175/175 [==============================] - 20s 104ms/step - loss: 1.3156
Epoch 8/10
175/175 [==============================] - ETA: 0s - loss: 1.2699

175/175 [==============================] - 16s 78ms/step - loss: 1.2699
Epoch 9/10
175/175 [==============================] - ETA: 0s - loss: 1.2289

175/175 [==============================] - 15s 75ms/step - loss: 1.2289
Epoch 10/10
175/175 [==============================] - ETA: 0s - loss: 1.1870

175/175 [==============================] - 16s 81ms/step - loss: 1.1870
CPU times: user 2min 36s, sys: 4.81 s, total: 2min 41s
Wall time: 2min 56s


## Generate Text
The simplest way to generate text with this model is to run it in a loop, and keep track of the model's internal state as we execute it.

Each time we call the model we pass in some text and an internal state. The model returns a prediction for the next character and its new state. Pass the prediction and state back in to countinue generating text.

In [ ]:
class OneStep(tf.keras.Model):
    def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
        super().__init__()
        self.temperature = temperature
        self.model = model
        self.chars_from_ids = chars_from_ids
        self.ids_from_chars = ids_from_chars

        # Create a mask to prevent "[UNK]" from being generated.
        skip_ids = self.ids_from_chars(["[UNK]"])[:, None]
        sparse_mask = tf.SparseTensor(
            # Put a -inf at each bad index.
            values=[-float("inf")] * len(skip_ids),
            indices=skip_ids,
            # Match the shape to the vocabulary
            dense_shape=[len(ids_from_chars.get_vocabulary())],
        )
        self.prediction_mask = tf.sparse.to_dense(sparse_mask)

    @tf.function
    def generate_one_step(self, inputs, states=None):
        # Convert strings to token IDs.
        input_chars = tf.strings.unicode_split(inputs, "UTF-8")
        input_ids = self.ids_from_chars(input_chars).to_tensor()

        # Run the model.
        # predicted_logits.shape is [batch, char, next_char_logits]
        predicted_logits, states = self.model(
            inputs=input_ids, states=states, return_state=True
        )
        # Only use the last prediction.
        predicted_logits = predicted_logits[:, -1, :]
        predicted_logits = predicted_logits / self.temperature
        # Apply the prediction mask: prevent "[UNK]" from being generated.
        predicted_logits = predicted_logits + self.prediction_mask

        # Sample the output logits to generate token IDs.
        predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
        predicted_ids = tf.squeeze(predicted_ids, axis=-1)

        # Convert from token ids to characters
        predicted_chars = self.chars_from_ids(predicted_ids)

        # Return the characters and model state.
        return predicted_chars, states

In [ ]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

Run it in a loop to generate some text. Looking at the genereaated text, you'll see that model knows when to capitalize, make paragraps and imitates a Shakespare-like writing vocabulary. With the small number of training epochs, it has not yet learne to form coherent sentences.

In [ ]:
start = time.time()
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(1000):
    next_char, states = one_step_model.generate_one_step(
        next_char, states = states
    )
    result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), "\n\n" + "_" * 80)
print("\nRun time:", end - start)

ROMEO:
After our maintage of fair walls, not that they
Batt, possess the remorwhal,
Denied the Bolingbroke, do leave upon me A begganant
May hence; and wetch of yestend
But the most speech of mine oraward!

CORIOLANUS:
Because to keep your lady's doubhent.

POLIXENSE:
Who thou that bid me asswer mine inteem sprin,
Or in my love behind thy beast with thee?

KING RICHARD III:
And learn their sue that please your grace:
The perilloth was the valley, thither in half
To straw thy will. Gentlemen, call Hall!

Third Citizen:
Not boin: and Montague,
You dance what thou dost calm Cavil withmit thee:
The headsine royal elpowers come?
Ha! you be juiced in our master Antigonue,
That hath woundshmance: be cross'd privately, you
hither to her fairer with the feast wails Lecembard.

LEONTES:
The dure groos sidely be Mesel'd,
And all thy earl of Heruly, and I think I am more ancher.

CORIOLANUS:
For this, I came both
To go in our turn; we'll divide this old:
The field with me both made the false polic

The easies thing you can do to imporve the results is to trian it for longer (try `EPOCHS = 30`).

We can also experiment with diffenret start stirng, try adding another RNN layer to imporve the model's accuracy, or adjust the temperature parameter to generate more or less random predictons.

If we want the model to generate text faster the easiest thing we can do is batch the next generation in the example below the model generates 5 output in about the same time it took to generate 1 above

In [ ]:
start = time.time()
states = None
next_char = tf.constant(['ROMEO', 'ROMEO', 'ROMEO', 'ROMEO', 'ROMEO' ])
result = [next_char]

for n in range(1000):
    next_char, states = one_step_model.generate_one_step(
        next_char, states = states
    )
    result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result, '\n\n' + "_" * 80)
print("\nRun time", end - start)

tf.Tensor(
[b"ROMEO:\nNot to the County Pire here, sir, if Wert me not\nAnd let your brother's worse.\n\nLUCENTIO:\nWhat is't not the country: we changed spite!\nCome to your weeping, and my name be plainment\nAnd dear sons weally: back him Romeo in brother,\nAnother drownting against the earth;\nA troophood any monaward of Madgeal spirit,\nAnd creph in so secrew'd him, she doubt her unbeg,\nThe want of England stives are we remeth\nWith which and every godness from deep in them;\nAnd pity her town what't in haste,\nThat our king great kins without your hands,\nMay chanced in your noteous to our people.\n\nANGELO:\nClaudio; for Gerries and be bliss.\nBut let me let embrech's thy deadife!\n\nBENVOLIO:\nWhat is the haste for while we doad, like a food?\n\nHASTINGS:\nWhy none of the maid, my weven wants\nThat then us does me; for tell the presence challen\nCall home.\n\nBUCKINGHAM:\nYour grace is mady to be nor wrick'd in coronation.\nDurpe it is to earnesh; cold son, you have some crown,

## Export the generator
This signal-step nodel can easily be saved and restored, allowing you to use it anywhere a `tf.saved_model` is accepted

In [ ]:
# save the one step model
tf.saved_model.save(one_step_model, 'one_step')

In [ ]:
# reload the model
one_step_reloaded = tf.saved_model.load('one_step')

In [ ]:
states =  None
next_char = tf.constant(['ROMEO'])
result = [next_char]

for n in range(100):
    next_char, states = one_step_reloaded.generate_one_step(
        next_char, states = states
    )
    result.append(next_char)

print(tf.strings.join(result)[0].numpy().decode("utf-8"))

ROMEO:
Dogs match, and 'maritom'd you, if my sole duke the world,
That she your promises: come upon that 


## Advanced: Customized Training

The above training procedure is simple, but it does not give us much control, it uses teacher-forcing which prevents bad predicitons from being fed back to the model, so the model never learns to recover form mistakes.

So now that we've seen how to run the model manualy next we can implement the tarining loop. This gives a starting point if, for example, we want to implement curriculum learning to help stabilize the model's open-loop output.

The most important part of a custom training loop is the train step function

We use `tf.GradientTape` to track the gradients. You can learn more about this approach by reading the [eager execution guide](https://www.tensorflow.org/guide/basics).

The basic procedure is

1. Execute the model and calculate the loss under a `tf.GradientTape`.
2. Calculate the updates and aply them to the model using optimizer.


In [ ]:
class CustomTraining(MyModel):
    @tf.function
    def train_step(self, inputs):
        inputs, labels = inputs
        with tf.GradientTape() as tape:
            predictions = self(inputs, training=True)
            loss = self.loss(labels, predictions)
        grads = tape.gradient(loss, model.trainable_variables)
        self.optimizer.apply_gradients(zip(grads, model.trainable_variables))

        return {'loss': loss}

The above implementation of the `train_step` method follows `Keras train_step conventions`. This is optional, but it allows us to change the begavior of the train step and still use keras `Model.cmpile` and `Model.fit` methods.

In [ ]:
model = CustomTraining(
    vocab_size = len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units = rnn_units
)

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
)

In [ ]:
model.fit(dataset, epochs=1)

175/175 [==============================] - 15s 57ms/step - loss: 2.7085
